## A few examples hinting at web integrations and data analysis

Import the libraries we are going to use

In [ ]:
import sqlite3
import pandas as pd
import ipywidgets

An example of interactivity with this web page

In [ ]:
def f(x):
    return x

In [ ]:
ipywidgets.interact(f,x=(0,10));

From previously, we establish the connection object to the database

In [ ]:
conn = sqlite3.connect('survey.db')

and establish the cursor object for executing queries

In [ ]:
cur = conn.cursor()

and use the cursor to execute queries and return results

In [ ]:
cur.execute("select * from person;")
for i in cur.fetchall():
    print(i)

Or use a wrapper and Pandas to query similarly

In [ ]:
def run_query_pd(query,connection):
    return pd.read_sql_query(query,connection)

In [ ]:
run_query_pd('select name from sqlite_master;',conn)

In [ ]:
query = '''
SELECT *
FROM person;
'''
run_query_pd(query,conn)

Let's incorporate the interactivity with the SQL query

In [ ]:
def g(x):
    
    query = '''
    SELECT *
    FROM person
    LIMIT {:d};
    '''.format(x)
    
    return run_query_pd(query,conn)

ipywidgets.interact(g,x=(0,5));

In [ ]:
# close the cursor and connection normally, but leave open here so Voila works ok
# just be careful that later connections either are or are not named similarly
# depending on how you want to use them
#cur.close()
#conn.close()

## Use an external web page to interact with SQL and the database

In [ ]:
connection = sqlite3.connect('survey.db')

### Interactive Add

In [ ]:
def addname(nameid,personal,last):
    query = "INSERT INTO Person (id, personal, family) VALUES (?, ?, ?);"

    cursor = connection.cursor()
    cursor.execute(query, [nameid,personal,last])
    cursor.close()

    connection.commit()

    query = 'SELECT * FROM person;'

    return run_query_pd(query,connection)

In [ ]:
ipywidgets.interact_manual(addname,
                           nameid=ipywidgets.Text(),
                           personal=ipywidgets.Text(),
                           last=ipywidgets.Text());

### Interactive delete

In [ ]:
def deleteid(nameid):
    query = "DELETE FROM Person WHERE id = (?);"

    cursor = connection.cursor()
    cursor.execute(query, [nameid])
    cursor.close()

    connection.commit()

    query = 'SELECT * FROM person;'
    newview = pd.read_sql_query(query,connection)  

    return run_query_pd(query,connection)

In [ ]:
ipywidgets.interact_manual(deleteid,
                           nameid=ipywidgets.Text());